In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import time
from tqdm import tqdm

In [4]:
from selenium import webdriver

In [ ]:
def musinsa_rank(category_num,page_num):
    url = f"https://www.musinsa.com/ranking/best?period=now&age=ALL&mainCategory=00{category_num}&subCategory=&leafCategory=&price=&golf=false&kids=false&newProduct=false&exclusive=false&discount=false&soldOut=false&page={page_num}&viewType=small&priceMin=&priceMax="
    response = requests.get(url)
    html = bs(response.text, 'lxml')
    musinsa_rank_df = rbnl(html)

    return musinsa_rank_df

In [8]:
def rbnl(html):
    musinsa_rank_df = pd.DataFrame()

    #순위 뽑기
    rank_html = html.select('#goodsRankList > li > p')
    rank_no_list = []

    for i in rank_html:
        rank_no_list.append(i.string.strip())

    musinsa_rank_df['순위'] = rank_no_list

    #브랜드 이름 뽑기
    brand_html = html.select('#goodsRankList > li > div.li_inner > div.article_info > p.item_title > a')
    brand_list = []

    for i in brand_html:
        brand_list.append(i.string)

    musinsa_rank_df['브랜드명']=brand_list

    #링크와 의류명 뽑기
    link_name_html = html.select('#goodsRankList > li > div.li_inner > div.article_info > p.list_info > a')
    link_list = []
    name_list = []

    for i in link_name_html:
        link_list.append(i['href'])
        name_list.append(i['title'])

    musinsa_rank_df['의류명']=name_list
    musinsa_rank_df['링크']=link_list

    #상세 페이지 크롤링
    musinsa_rank_df2 = specific_info(link_list)

    #데이터 프레임 옆으로 합치기
    musinsa_rank_df = pd.concat([musinsa_rank_df, musinsa_rank_df2], axis=1)

    return musinsa_rank_df

In [7]:
def specific_info(link_list):
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'}

    musinsa_rank_df = pd.DataFrame()
    part_num_list=[]
    sex_list=[]
    view_list=[]
    sales_list=[]
    like_list=[]


    for link in tqdm(link_list):
#         print(link)
        response_1=requests.get(link,headers=headers)
        html_1=bs(response_1.text, 'lxml')

        #품번 리스트 생성
        part_num_html=html_1.select('#product_order_info > div.explan_product.product_info_section > ul > li:nth-child(1) > p.product_article_contents > strong')
        part_num=part_num_html[0].get_text().split('/')[-1].strip()
        part_num_list.append(part_num)

        #성별 리스트 생성
        sex_html=html_1.select("#product_order_info > div.explan_product.product_info_section > ul > li > p.product_article_contents > span.txt_gender")
        sex=sex_html[0].get_text().replace('\n',' ').strip()
        sex_list.append(sex)

        #셀레니움으로 원하는 데이터 가져오기
        driver = webdriver.Chrome('chromedriver.exe')
        driver.get(link)
        sel_html=driver.page_source
        html_2=bs(sel_html)

        #조회수 가져오기
        view_html=html_2.find_all("strong", {"id":"pageview_1m"})
        view=view_html[0].get_text()
        view_list.append(view)

        #누적 판매 가져오기
        sales_html=html_2.find_all("strong", {"id":"sales_1y_qty"})
        sales=sales_html[0].get_text()
        sales_list.append(sales)

        #좋아요 수 가져오기
        like_html=html_2.find_all("span", {"class": "prd_like_cnt"})
        like=like_html[0].get_text()
        like_list.append(like)

        #드라이버 닫아주기
        driver.close()

        #시간 추가
        time.sleep(0.01)

    musinsa_rank_df['품번']=part_num_list
    musinsa_rank_df['성별']=sex_list
    musinsa_rank_df['조회수']=view_list
    musinsa_rank_df['누적판매량(1년)']=sales_list
    musinsa_rank_df['좋아요']=like_list

    return musinsa_rank_df

## 파일로 저장

In [13]:
category_num=input("Category 번호를 입력하세요: ")
page_num=input("Page 번호를 입력하세요: ")
final_df=musinsa_rank(category_num,page_num)

  0%|          | 0/90 [00:00<?, ?it/s]C:\Users\jpc10\AppData\Local\Temp\ipykernel_37940\2940746359.py:28: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')
100%|██████████| 90/90 [13:58<00:00,  9.31s/it]


In [14]:
file_name = f"musinsa_ranking_category{category_num}_page{page_num}.csv"
final_df.to_csv(file_name, index=False)
pd.read_csv(file_name)

,순위,브랜드명,의류명,링크,품번,성별,조회수,누적판매량(1년),좋아요
0,181위,퍼스텝,리메인 바시티 자켓 블랙/크림 JUOT4536,https://www.musinsa.com/app/goods/3028088?loc=...,5001963592,남,4.5만 회 이상,500개 이상,"6,155"
1,182위,논플로어,윈드 테이커 자켓 7.COL,https://www.musinsa.com/app/goods/2732486?loc=...,11832746,남 여,3.2만 회 이상,1.1천 개 이상,"4,723"
2,183위,애프터프레이,카펜터 데님 워크 재킷 라벤더,https://www.musinsa.com/app/goods/3075400?loc=...,2110013000022,남,200회 이상,NaN,23
3,184위,빠니깔레,윈터 스판 코튼 D재킷 [2COLOR],https://www.musinsa.com/app/goods/2864075?loc=...,D-ACJK63,남,NaN,NaN,9
4,185위,로라로라,V NECK CABLE CARDIGAN NAVY,https://www.musinsa.com/app/goods/2719743?loc=...,5000026935,여,2.9만 회 이상,4.9천 개 이상,"26,153"
...,...,...,...,...,...,...,...,...,...
85,266위,레프트서울,리얼 양가죽 베지터블 오버핏 자켓 [브라운],https://www.musinsa.com/app/goods/3107627?loc=...,DMY001BRN,남,NaN,NaN,25
86,267위,드로우핏,오버사이즈 미니멀 체크 블레이저 [BEIGE],https://www.musinsa.com/app/goods/709192?loc=g...,DFJK0002BE,남,4.9천 회 이상,400개 이상,"6,457"
87,268위,파브레가,범피 울 히든 블루종 자켓 (미드 그레이),https://www.musinsa.com/app/goods/2859173?loc=...,Wool Bumpy Blouson Jacket (Mid Grey),남,1만 회 이상,200개 이상,"1,058"
88,269위,타미힐피거,풀 밀라노 릴렉스 가디건 (T12C6KCD020MT2BDS),https://www.musinsa.com/app/goods/2700919?loc=...,T12C6KCD020MT2BDS,남,1.7천 회 이상,50개 이상,331
